# Pentathlon-III: Next Product to Buy Models

* Team-lead GitLab userid: 243
* Group name: Korinna
* Team member names: Siqi Chen, Wen-Hsuan Hung, Xinyu Lou, Yuefeng Mao

## Setup

Please complete this python notebook with your group by answering the questions in `pentathlon-nptb.pdf`. Create a Notebook and HTML file with all your results and comments and push both the Notebook and HTML file to GitLab when your team is done. All results MUST be reproducible (i.e., the TA and I must be able to recreate the HTML file from the Jupyter Notebook without changes or errors). This means that you should NOT use any python-packages that are not part of the rsm-msba-spark docker container.

This is the third group assignment for MGTA 455 and you will be using Git and GitLab. If two people edit the same file at the same time you could get what is called a "merge conflict". This is not something serious but you should realize that Git will not decide for you who's change to accept so the team-lead will have to determine the edits to use. To avoid merge conflicts, **always** "pull" changes to the repo before you start working on any files. Then, when you are done, save and commit your changes, and then push them to GitLab. Make "pull first" a habit!

If multiple people are going to work on the assignment at the same time I recommend you work in different notebooks. You can then `%run ...`  these "sub" notebooks from the main assignment file. You can seen an example of this in action below for the `model1.ipynb` notebook

Some group work-flow tips:

* Pull, edit, save, stage, commit, and push
* Schedule who does what and when
* Try to avoid working simultaneously on the same file 
* If you are going to work simultaneously, do it in different notebooks, e.g., 
    - model1.ipynb, model2.ipynb, model3.ipynb
* Use the `%run ... ` command to bring different pieces of code together into the main jupyter notebook
* Put python functions in modules that you can import from your notebooks. See the example below for the `example` function defined in `utils/functions.py`

A graphical depiction of the group work-flow is shown below:

![](images/git-group-workflow-wbg.png)

Tutorial videos about using Git, GitLab, and GitGadget for group assignments:

* Setup the MSBA server to use Git and GitLab: https://youtu.be/zJHwodmjatY
* Dealing with Merge Conflicts: https://youtu.be/qFnyb8_rgTI
* Group assignment practice: https://youtu.be/4Ty_94gIWeA

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyrsm as rsm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import  OneHotEncoder

from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report, roc_curve, auc, make_scorer,mean_squared_error,r2_score


import matplotlib.pyplot as plt
# increase plot resolution
# mpl.rcParams["figure.dpi"] = 150

In [3]:
## loading the data - this dataset must NOT be changed
pentathlon_nptb = pd.read_pickle("data/pentathlon_nptb.pkl")
pentathlon_nptb["buyer_yes"] = (pentathlon_nptb["buyer"] == "yes").astype(int)
pentathlon_nptb.head()

,custid,buyer,total_os,message,age,gender,income,education,children,freq_endurance,...,endurance_os,strength_os,water_os,team_os,backcountry_os,winter_os,racquet_os,training,representative,buyer_yes
0,U45198803,no,0.0,endurance,30 to 44,M,25000,14,1.3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2147483648,1,0
1,U22197752,no,0.0,backcountry,45 to 59,F,40000,44,0.4,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2147483648,1,0
2,U83874832,no,0.0,backcountry,45 to 59,M,50000,24,0.8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0
3,U19423462,no,0.0,winter,45 to 59,F,50000,26,1.1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2147483648,1,0
4,U23888305,no,0.0,winter,30 to 44,M,40000,22,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2147483648,1,0


In [4]:
# show dataset description
rsm.describe(pentathlon_nptb)

## Pentathon III: Next Product To Buy 

The available data is based on the last e-mail sent to each Pentathlon customer. Hence, an observation or row in the data is a "customer-promotional e-mail" pair. The data contains the following basic demographic information available to Pentathlon:

* "age": Customer age(coded in 4 buckets:"<30", "30 to 44", "45 to 59", and ">=60") 
* "gender": Gender coded as F or M
* "income": Income in Euros, rounded to the nearest €5,000
* "education": Percentage of college graduates in the customer's neighborhood, coded from 0-100
* "children": Average number of children in the customer's neighborhood

The data also contains basic historical information about customer purchases, specifically, a department-specific frequency measure.

* "freq_endurance-freq_racquet": Number of purchases in each department in the last year, excluding any purchase in response to the last email.

The key outcome variables are:

* "buyer": Did the customer click on the e-mail and complete a purchase within twodays of receiving the e-mail ("yes" or "no")?
* "total_os": Total order size (in Euros) conditional on the customer having purchased (buyer == "yes"). This measures spending across all departments, not just the department that sent the message

While of no importance for the prediction model, we also have data on which departments the customer ordered from when they purchased. This was captured in:

* "endurance_os - racquet_os": Department-specific order size (in Euros). This was a breakdown of the total order size if buyer == "yes". The value was zero for most departments because customers rarely bought products from multiple departments on a single purchase occasion.

Finally, Anna requested that her team divide the data into a training sample and a test sample using a 70 - 30 split. She suggested 70,000 observations in the training sample and 30,000 observations in the test sample. Anna requested the team sample more customers who purchased relative to those that did not purchase. She had learned that some analytical tools did a better job of scoring customers if the response variable had a similar number of "yes" and "no" values.

To achieve a 50/50 split between buyers and non-buyers for the training and test sample, her team randomly picked 50,000 buyers and added 50,000 randomly sampled non-buyers. Given the actual average purchase rate of 1%, buyers in this training sample were oversampled by a factor of 50 (=0.5/0.01). The 100,000 customers were then randomly split into a training sample (70,000 customers) and a test sample (30,000 customers).

In addition to the 100,000 customers used for training and test, Anna asked the team to add a representative sample consisting of another 100,000 customers. This sample was representative in that it was a true random sample of the population and therefore contained the average proportion of buyers, namely 1%. This sample would be used to determine the expected benefits from using a next-product-to-buy model. In summary, the dataset contains 200,000 customers.

1. 70,000 in a training sample (training == 1) 
2. 30,000 in a test sample (training == 0)
3. 100,000 in a representative sample (use is.na(training) or representative == 1)

In [5]:
# run python code from another notebook
%run ./sub-notebooks/model1.ipynb

                    index     OR     OR%   2.5%  97.5% p.values     
1     message[T.strength]  0.977   -2.3%  0.925  1.033    0.415     
2        message[T.water]  0.937   -6.3%  0.887  0.991    0.022    *
3         message[T.team]  0.873  -12.7%  0.826  0.922   < .001  ***
4  message[T.backcountry]  0.859  -14.1%  0.812  0.908   < .001  ***
5       message[T.winter]  0.875  -12.5%  0.828  0.925   < .001  ***
6      message[T.racquet]  0.874  -12.6%  0.826  0.923   < .001  ***


In [6]:
# importing functions from a module/package
from utils import functions
functions.example()


You just accessed a function from your first python packages!
Change the code in utils/function.py to whatever you need for this assignment
Use 'from utils import functions' to get access to your code
You can add modules to import from by adding additional .py files to the 'utils' directory
Note: If you make changes to the content of this file you will have to restart the notebook kernel to get the updates



In [7]:
pd.options.mode.chained_assignment = None

# Part1 Model Selection

In [5]:
pentathlon_nptb["age"] =  pentathlon_nptb.age.astype('category')
pentathlon_nptb["gender"] =  pentathlon_nptb.gender.astype('category')
pentathlon_nptb["message"] =  pentathlon_nptb.message.astype('category')
pentathlon_nptb["buyer_yes"] = rsm.ifelse(pentathlon_nptb.buyer == "yes", 1, rsm.ifelse(pentathlon_nptb.buyer == "no", 0, np.nan))

to_std = pentathlon_nptb.loc[:, "income":"freq_racquet"].columns

# scale numeric variables by (x - mean(x)) / sd(x)
pentathlon_nptb_std = pentathlon_nptb.copy()
pentathlon_nptb_std[to_std] = rsm.scale_df(
    pentathlon_nptb[to_std], sf=1
)

In [6]:
pentathlon_train = pentathlon_nptb_std[pentathlon_nptb_std['training'] == 1]
pentathlon_test = pentathlon_nptb_std[pentathlon_nptb_std['training'] == 0]

## A. logistic regression

In [8]:
lr = smf.glm(
    formula="buyer_yes ~ message + age + gender + income + education + children + \
             freq_racquet + freq_winter + freq_backcountry + freq_team + \
             freq_water + freq_strength + freq_endurance + message:age + message:gender + message:income + message:education + message:children + \
             message:freq_racquet + message:freq_winter + message:freq_backcountry + \
             message:freq_team + message:freq_water + message:freq_strength + message:freq_endurance",
    family=Binomial(link=logit()),
    data=pentathlon_train,
).fit()

In [10]:
pentathlon_test["pred_logit"] = lr.predict(pentathlon_test)

In [10]:
def score_fun(y_pred,y_true):
    AUC = roc_auc_score(y_true,y_pred)
    R2 =r2_score(y_true, y_pred)
    RMSE = np.sqrt(mean_squared_error(y_true, y_pred))
    
    print('AUC ' + str(AUC))
    print('R2 ' + str(R2))
    print('RMSE ' + str(RMSE))
    
    return AUC,R2,RMSE

In [11]:
AUC_Logist, R2_Logist, RMSE_Logist = score_fun(pentathlon_test["pred_logit"],pentathlon_test["buyer_yes"])

AUC 0.8830857777777779
R2 0.450442046599711
RMSE 0.3706608805229819


#### Processing data for other models

In [11]:
### one hot
enc = OneHotEncoder()
enc.fit(pentathlon_nptb_std.loc[:,"message":"gender"])
ans= enc.transform(pentathlon_nptb_std.loc[:,"message":"gender"]).toarray()
ans = pd.DataFrame(ans)
X_num = pentathlon_nptb_std.loc[:,"income":"freq_racquet"]
X_index = pentathlon_nptb_std.loc[:,"training":"representative"]
X = pd.concat([ans,X_num,X_index],axis=1)

In [12]:
X_train = X[X['training']==1]
X_test = X[X['training']==0]
X_repre = X[X['representative']==1]

In [13]:
Y_train = pentathlon_nptb_std[pentathlon_nptb_std['training']==1]["buyer_yes"]
Y_test = pentathlon_nptb_std[pentathlon_nptb_std['training']==0]["buyer_yes"]
Y_repre = pentathlon_nptb_std[pentathlon_nptb_std['representative']==1]["buyer_yes"]

## B. GradientBoosting

In [15]:
learning_rate = list(np.arange(0.2,0.6,0.05))
n_estimators = range(10,110,10)
max_depth = range(1,5)

def gridsearch_gb(learning_rate,n_estimators,max_depth):
    params_scores = {}
    for j in n_estimators:
        for k in max_depth:
            for g in learning_rate:
                model = GradientBoostingClassifier(n_estimators=j,max_depth=k,learning_rate=g).fit(X_train,Y_train)
                y_pred = model.predict_proba(X_test)[:,1]
                score = roc_auc_score(Y_test, y_pred)
                params_scores[(j,k,g)] = score
            print("n_estimators=",j,"max_depth=",k,"learning_rate=",g," AUROC on testing set=",score)
    return max(params_scores, key=params_scores.get)

gridsearch_gb(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate)

n_estimators= 10 max_depth= 1 learning_rate= 0.5499999999999999  AUROC on testing set= 0.8652620777777777
n_estimators= 10 max_depth= 2 learning_rate= 0.5499999999999999  AUROC on testing set= 0.8767077888888889
n_estimators= 10 max_depth= 3 learning_rate= 0.5499999999999999  AUROC on testing set= 0.8785995755555555
n_estimators= 10 max_depth= 4 learning_rate= 0.5499999999999999  AUROC on testing set= 0.8824924755555557
n_estimators= 20 max_depth= 1 learning_rate= 0.5499999999999999  AUROC on testing set= 0.8734543200000001
n_estimators= 20 max_depth= 2 learning_rate= 0.5499999999999999  AUROC on testing set= 0.8812932911111111
n_estimators= 20 max_depth= 3 learning_rate= 0.5499999999999999  AUROC on testing set= 0.8834431311111112
n_estimators= 20 max_depth= 4 learning_rate= 0.5499999999999999  AUROC on testing set= 0.8861929955555555
n_estimators= 30 max_depth= 1 learning_rate= 0.5499999999999999  AUROC on testing set= 0.876435131111111
n_estimators= 30 max_depth= 2 learning_rate= 0.

(100, 4, 0.2)

In [16]:
gb = GradientBoostingClassifier(learning_rate=0.2, max_depth=4, n_estimators=100).fit(X_train,Y_train)
# Evaluate test set 
pentathlon_test["pred_Boosting"] = gb.predict_proba(X_test)[:,1]
AUC_Boosting, R2_Boosting, RMSE_Boosting = score_fun(pentathlon_test["pred_Boosting"],pentathlon_test["buyer_yes"])

AUC 0.8878554488888889
R2 0.46098464685065244
RMSE 0.3670883249128701


<ipython-input-16-fd8f1005e103>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pentathlon_test["pred_Boosting"] = gb.predict_proba(X_test)[:,1]


## C. XGB

In [17]:
learning_rate = list(np.arange(0.2,0.6,0.05))
n_estimators = range(10,110,10)
max_depth = range(1,5)

def gridsearch_Xgb(learning_rate,n_estimators,max_depth):
    params_scores = {}
    for j in n_estimators:
        for k in max_depth:
            for g in learning_rate:
                model = XGBClassifier(n_estimators=j,max_depth=k,learning_rate=g).fit(X_train,Y_train)
                y_pred = model.predict_proba(X_test)[:,1]
                score = roc_auc_score(Y_test, y_pred)
                params_scores[(j,k,g)] = score
    print("n_estimators=",j,"max_depth=",k,"learning_rate=",g," AUROC on testing set=",score)
    return max(params_scores, key=params_scores.get)

gridsearch_Xgb(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate)

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:40:36] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.8/dist-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.8/dist-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassi

[22:40:36] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:40:37] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.8/dist-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[22:44:49] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.8/dist-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:44:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
n_estimators= 100 max_depth= 4 learning_rate= 0.5499999999999999  AUROC on testing set= 0.8851402466666667


/usr/local/lib/python3.8/dist-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


(100, 4, 0.2)

In [18]:
Xgb = XGBClassifier(learning_rate=0.2, max_depth=4, n_estimators=100).fit(X_train,Y_train)
# Evaluate test set 
pentathlon_test["pred_XGB"] = gb.predict_proba(X_test)[:,1]
AUC_Xgb, R2_Xgb, RMSE_Xgb= score_fun(pentathlon_test["pred_XGB"],pentathlon_test["buyer_yes"])

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:44:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
AUC 0.8878554488888889
R2 0.46098464685065244
RMSE 0.3670883249128701


<ipython-input-18-8380a482179c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pentathlon_test["pred_XGB"] = gb.predict_proba(X_test)[:,1]


## D. Neural Network

In [19]:
# define grid
alpha = list(np.arange(0.01,0.1,0.01))
hidden_layer_sizes = [(8,),(10,),(8,8),(10,10)]

def gridsearch_nn(alpha, hidden_layer_sizes):
    params_scores = {}
    for j in alpha:
        for k in hidden_layer_sizes:
            model = MLPClassifier(alpha=j, hidden_layer_sizes=k, learning_rate_init=0.5,max_iter=10000).fit(X_train,Y_train)
            y_pred = model.predict_proba(X_test)[:,1]
            score = roc_auc_score(Y_test, y_pred)
            params_scores[(j,k)] = score
            print("alpha=",j,"hidden_layer_sizes=",k," AUROC on testing set=",score)
    return max(params_scores, key=params_scores.get)

gridsearch_nn(alpha=alpha, hidden_layer_sizes=hidden_layer_sizes)

alpha= 0.01 hidden_layer_sizes= (8,)  AUROC on testing set= 0.8821896800000001
alpha= 0.01 hidden_layer_sizes= (10,)  AUROC on testing set= 0.8813898155555556
alpha= 0.01 hidden_layer_sizes= (8, 8)  AUROC on testing set= 0.5
alpha= 0.01 hidden_layer_sizes= (10, 10)  AUROC on testing set= 0.5
alpha= 0.02 hidden_layer_sizes= (8,)  AUROC on testing set= 0.8762158266666668
alpha= 0.02 hidden_layer_sizes= (10,)  AUROC on testing set= 0.8803522622222223
alpha= 0.02 hidden_layer_sizes= (8, 8)  AUROC on testing set= 0.7981691733333334
alpha= 0.02 hidden_layer_sizes= (10, 10)  AUROC on testing set= 0.5
alpha= 0.03 hidden_layer_sizes= (8,)  AUROC on testing set= 0.8792940844444443
alpha= 0.03 hidden_layer_sizes= (10,)  AUROC on testing set= 0.8790703066666666
alpha= 0.03 hidden_layer_sizes= (8, 8)  AUROC on testing set= 0.8750325244444445
alpha= 0.03 hidden_layer_sizes= (10, 10)  AUROC on testing set= 0.5
alpha= 0.04 hidden_layer_sizes= (8,)  AUROC on testing set= 0.8818313866666667
alpha= 0.04 

(0.05, (8, 8))

In [20]:
best_model_NN = MLPClassifier(alpha=0.07, hidden_layer_sizes=(8,),learning_rate_init=0.5, max_iter=10000).fit(X_train,Y_train)
# Evaluate test set 
pentathlon_test["pred_NN"] = best_model_NN.predict_proba(X_test)[:,1]
AUC_NN, R2_NN, RMSE_NN = score_fun(pentathlon_test["pred_NN"],pentathlon_test["buyer_yes"])

AUC 0.8797045155555556
R2 0.3684874040515218
RMSE 0.39733883397815467


<ipython-input-20-a75cbd6225a7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pentathlon_test["pred_NN"] = best_model_NN.predict_proba(X_test)[:,1]


## E. Random Forest

In [21]:
n_estimators=[141,142,143,145,150,160,170]
max_depth=[11,12,13,14,15,16,17]

def gridsearch_rf(n_estimators,max_depth):
    params_scores = {}
    for j in n_estimators:
        for k in max_depth:
            model = RandomForestClassifier(n_estimators=j,max_depth=k).fit(X_train,Y_train)
            y_pred = model.predict_proba(X_test)[:,1]
            score = roc_auc_score(Y_test, y_pred)
            params_scores[(j,k)] = score
            print("n_estimators=",j,"max_depth=",k," AUROC on testing set=",score)
    return max(params_scores, key=params_scores.get)

gridsearch_rf(n_estimators=n_estimators,max_depth=max_depth)

n_estimators= 141 max_depth= 11  AUROC on testing set= 0.8861723266666666
n_estimators= 141 max_depth= 12  AUROC on testing set= 0.8864378777777776
n_estimators= 141 max_depth= 13  AUROC on testing set= 0.88656082
n_estimators= 141 max_depth= 14  AUROC on testing set= 0.8869787333333334
n_estimators= 141 max_depth= 15  AUROC on testing set= 0.8864501622222222
n_estimators= 141 max_depth= 16  AUROC on testing set= 0.8857872066666665
n_estimators= 141 max_depth= 17  AUROC on testing set= 0.8852653133333335
n_estimators= 142 max_depth= 11  AUROC on testing set= 0.8859161155555555
n_estimators= 142 max_depth= 12  AUROC on testing set= 0.8867029955555555
n_estimators= 142 max_depth= 13  AUROC on testing set= 0.8865695688888889
n_estimators= 142 max_depth= 14  AUROC on testing set= 0.8866899244444445
n_estimators= 142 max_depth= 15  AUROC on testing set= 0.8864541422222222
n_estimators= 142 max_depth= 16  AUROC on testing set= 0.8860297799999999
n_estimators= 142 max_depth= 17  AUROC on test

(141, 14)

In [22]:
best_model_RF = RandomForestClassifier(n_estimators = 170, max_depth = 14).fit(X_train, Y_train)
pentathlon_test["pred_RF"] = best_model_RF.predict_proba(X_test)[:,1]

AUC_RF, R2_RF, RMSE_RF = score_fun(pentathlon_test["pred_RF"],pentathlon_test["buyer_yes"])

AUC 0.8866553177777778
R2 0.45792672918854105
RMSE 0.36812812674782774


<ipython-input-22-b425d496b527>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pentathlon_test["pred_RF"] = best_model_RF.predict_proba(X_test)[:,1]


## F. Decision Tree

In [23]:
max_depth= list(range(150,160,5))
min_samples_leaf= list(range(3,5,1))
min_samples_split= list(np.arange(0.01,0.05,0.01))
max_features= list(range(15,20,1))


def gridsearch_dt(min_samples_leaf,min_samples_split,max_depth,max_features):
    params_scores = {}
    for j in min_samples_leaf:
        for k in min_samples_split:
            for g in max_depth:
                for l in max_features:
                    model = DecisionTreeClassifier(min_samples_leaf=j,min_samples_split=k,max_depth=g,max_features=l).fit(X_train,Y_train)
                    y_pred = model.predict_proba(X_test)[:,1]
                    score = roc_auc_score(Y_test, y_pred)
                    params_scores[(j,k,g,l)] = score
    
    print("min_samples_leaf =",j,"min_samples_split=",k,"max_depth=",g,'max_features = ',l," AUROC on testing set=",score)
    return max(params_scores, key=params_scores.get)

gridsearch_dt(max_depth=max_depth,min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split,max_features=max_features)


min_samples_leaf = 4 min_samples_split= 0.04 max_depth= 155 max_features =  19  AUROC on testing set= 0.869237088888889


(3, 0.01, 150, 19)

In [24]:
best_model = DecisionTreeClassifier(min_samples_leaf=3,min_samples_split=0.01,max_depth=150,max_features=19).fit(X_train,Y_train)
pentathlon_test["pred_DT"] = best_model.predict_proba(X_test)[:,1]

AUC_DT, R2_DT, RMSE_DT = score_fun(pentathlon_test["pred_DT"],pentathlon_test["buyer_yes"])

AUC 0.875527091111111
R2 0.4303391989851174
RMSE 0.37737938504073143


<ipython-input-24-d14b2dc6797e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pentathlon_test["pred_DT"] = best_model.predict_proba(X_test)[:,1]
